# ARCHIVO 1: Lectura y Procesamiento de Datos

In [1]:
#Importacion de las librerias a usar.
import requests
import numpy as np
import glob
import xlrd
import os
import sqlite3
from runSQL import *
import xlsxwriter 

### Get file Names:

Esta funcion se encarga de crear un archivo file.txt con el contenido html de la pagina de la ANH. Este es con el fin de buscar en el file.txt enlaces o las direcciones url de los archivos a descargar. Una vez encontradas y limpiadas retorna una lista con todos los url para descargar. 

In [2]:
def getFilesNames ():
    print("..................................................")
    print("Getting the name of the ANH files from the webpage")
    print("..................................................")
    #El url de la pagina anh.
    url = 'http://www.anh.gov.co/estadisticas-del-sector/sistemas-integrados-operaciones/estad%C3%ADsticas-producci%C3%B3n'
    
    #Se encarga de crear un archivo file.txt en la carpeta. El contenido de este archivo es el html de la pagina.
    html = requests.get(url)
    with open('file.txt', 'wb') as wr: 
        wr.write(html.content)
    wr.close()
    
    #Se abre el archivo file.txt
    read = open("file.txt","r+",encoding="utf-8")
    
    #Se encarga de leer cada linea del archivo hasta encontrar en que linea se encuentran los url de descarga de los archivos.
    htmlExcel = ""
    for line in read:
        xlsx = ".xlsx"
        if xlsx in line:
            htmlExcel = line
    #El codigo en el file.txt se ve asi: <>NombreDelArchivo<href="URL.xls"> entonces se quiere obtener el URL.xls
    #Se elimina los y se dejan espacios para separar el nombre del archivo con href= "URL.xls"
    htmlExcel.replace(">", " ")
    htmlExcel.replace("<", " ")
    #se crea la lista a retornar
    listaExcel = []
    #Se separan el href con el nombre del archivo
    separacionExcel= np.array(htmlExcel.split(" "))
    for i in range(0,len(separacionExcel)):
        if "href" in separacionExcel[i]:
            #aqui se separa con el href para solo tener el URL.xls
            l = separacionExcel[i].split("\"")
            #como es una extencion toca ponerle la pagina principal
            listaExcel.append("http://www.anh.gov.co"+l[1])
    read.close()
    #se elimina el archivo file.txt para no causar otros incovenientes:
    os.remove('file.txt')
    return listaExcel

## Download Files:

Esta funcción tiene como parametros los URL de cada archivo a descargar. Despues se encarga de limpiar estos datos para descargarlos y guardarlos con su nombre respectivo. Se descargan los archivos gases y crudo esto es para mantener libre la opcion de gases por si se requieren guardar en el futuro. Si no se quieren guardar con un if "crudo" in dls para solo guardar los de crudo.

In [3]:
def downloadFiles (listaExcel):
    print("downloading files... this might take some minutes")
    #Un recorrido de la lista listaExcel que contiene los URL a descargar
    for i in range(0, len(listaExcel)-1):
        #Aqui se encarga de descargar los archivos con el request
        dls = listaExcel[i]
        #aqui se pone para solo crudo:
        #if "Crudo" in dls or "crudo" in dls:
        resp = requests.get(dls)
        #Objetivo de esta parte es asignarle los nombres de cada archivo por lo tanto se limpian los datos
        name = dls.split("/")
        nombre = ""
        for j in range(0, len(name)):
            #Se encuentra en la direccion del url donde esta el nombre
            if ".xls" in name[j] or ".XLS" in name[j]:
                #Aqui se limpia el dado y se guarda
                nombre = name[j].split(".xls")[0]
                nombre = nombre.replace("%c3", "on")
                nombre = nombre.replace("b3n%", "")
                nombre = nombre.replace("%","-")
                nombre = nombre.replace(".","")
                nombre = nombre.replace("2020","20")
                #Hay dos formatos de excel uno viejo que es .xls y otro mas nuevo .xlsx se separan estos porque o sino genera
                #Un error cuando se lean.
                if len(name[j].split(".xls"))< 2:
                    nombre = nombre + ".xlsx"
                else:
                    if "x" in name[j].split(".xls")[1]:
                        nombre = nombre + ".xlsx"
                    else:
                        nombre = nombre + ".xls"
        #Se terminan de guardar con su nombre asignado y se cierra la función
        output = open(nombre, 'wb')
        output.write(resp.content)
        output.close()
    print("Download complete!")
        

## Get Gas Oil File Names:

Esta funcción tiene como objetivo obtener los nombres de los archivos Excel y retornarlos. Como parametro tiene una variable boolean que indica si se obtendran los datos recien descargados o localmente ubicados en la carpeta File. Como la ANH tiene ambos archivos gases y crudo se decidio hacer una lista de nombres para gases y crudo y una con ambos. Esto tiene como fin para dejar abierto la posibilidad de tratar los archivos gases tambien. Retorna lista de los nombres de archivos de gases, crudo y ambos.

In [4]:
def getGasOilFileNames(result):
    local = ""
    #Se chequea si es local o no. si es local se buscan los archivos en la carpeta Files.
    if result == False:
        local = "Files\\"
    directorio = local+"*.xlsx"
    directorio2 = local + "*.xls"
    #Busca y guarda los nombres de cada archivo que encuentra de tipo .xls y .xlsx
    listnamefiles = [f for f in glob.glob(directorio)]
    listnamefiles2 = [f for f in glob.glob(directorio2)]
    listnameGas=[]
    listnameOil=[]
    listnameAll = []
    
    #En esta separa los archivos de gases con los de crudo para los archivos con formato .xlsx
    #Los guarda en su lista correspondiente. 
    for i in range(0,len(listnamefiles)):
        if "Gas" in listnamefiles[i] or "gas" in listnamefiles[i]:
            listnameGas.append(listnamefiles[i])
        elif "Crudo" in listnamefiles[i] or "crudo" in listnamefiles[i]:
            listnameOil.append(listnamefiles[i])
        listnameAll.append(listnamefiles[i])
        
    #En esta separa los archivos de gases con los de crudo para los archivos con formato .xls  
    #Los guarda en su lista correspondiente.
    for i in range(0,len(listnamefiles2)):
        if "Gas" in listnamefiles2[i] or "gas" in listnamefiles2[i]:
            listnameGas.append(listnamefiles2[i])
        elif "Crudo" in listnamefiles2[i] or "crudo" in listnamefiles2[i]:
            listnameOil.append(listnamefiles2[i])
        listnameAll.append(listnamefiles2[i])
    return listnameGas, listnameOil, listnameAll 


## Encontrar Encabezado:

Tiene como función entrar a un archivo excel y encontrar el numero de fila que esta el encabezado. El encabezado es la parte del excel que muestra los titulos de cada columna como departamento, municipio, campo. Su paramentro es sheet que es la hoja del excel. Retorna el encabezado y el numero de fila que se encuentra.

In [5]:
def encontrarEncabezado(sheet):
    
    encabezado = []
    indiceE = 0
    #Recorrido para encontrar el numero de la fila del encabezado
    #se hace un recorrido de 100 porque el encabezado no suele estar en la fila 100 o mas.
    for i in range(0, 100):
        #Checkea si las primeras dos columnas no son nullas ya que suelen ser nulas con el formato dado
        #cuando ven que no son nulas es que ya encontro el encabezado.
        if sheet.row_values(i)[0]!=''and sheet.row_values(i)[1]!='':
            indiceE = i
            #sale de la busqueda
            break
    #Busca en el indice y guarda sus variables como: departamento municipio enero febrero entre otras
    #Este lo guarda en una lista llamada encabezado
    for i in range(0, len(sheet.row_values(indiceE))):
        t = sheet.row_values(indiceE)
       
        if t[i] == '':
            break
        else:
            encabezado.append(t[i])
    
    return encabezado, indiceE

## Encontrar Año

Tiene como objetivo encontrar el año del archivo. Esto se encuentran no solo en el nombre del archivo sino que tambien se encuentran adentro del archivo como titulo. Como el nombre del archivo en general tienen varios año en su nombre. Se prefiere usar el año del titulo.  Parametro el indice del encabezado y la hoja de excel, nombre del archivo. Retorna el numero de año.

In [6]:
def encontrarAnio(sheet, indiceE, name):
    
    encontrado = False
    i = 0
    j = 0
    valor= ""
    suma = 0
    #recorrido en cada fila y columna hasta encontrar un string que contenga 20
    while encontrado == False:

        valor = sheet.row_values(i)[j]
        #verifica si encontro el 20
        if "20" in valor:
            #si lo encontró entonces lo separa por el 20 y obtiene el numero que viene despues. 
            valor = valor.split("20")[1]
            valor = "20"+valor
            valor = valor.replace("*","")
            if valor == "20":
                #si es 20 entonces es 2020
                valor = "2020"
            encontrado= True
            break
        #Mover entre filas y despues entre columnas
        i+=1
        if i == indiceE:
            j+=1
            i=0
        suma = i + j
        #Para que solo mire en el nombre del archivo porque no hay en titulo
        if suma >= 1:
            break
    # si no se encontro en el titulo toca por el nomnbre de archivo
    if encontrado == False:  
        
        n = name
        numeros = ['1','2','3','4','5','7','8','9']
        #separar los nombres por 20 para ver donde estan
        n1 = n.split("20")
        lista = []
        for j in range(0, len(n1)):
            n2 = n1[j]
            n3 = list(n2)
            if len(n3) != 0:
                for s in range(0, len(numeros)):
                    #Verificar si es un numero porque aveces no son numeros sino algo como 20Crudo
                    if n3[0] == numeros[s]:
                        lista.append("20"+n3[0]+n3[1])
                        #Si es un numero guarde en lista y salga
                        break
        #Si no consiguio es porque es 2020 ya que el split elimino los 20
        if len(lista) == 0:
            valor = "2020"
        #si hay mas de un numero de 20 como 2015-2016 pues se agarra el minimo que es 2015 que comenzo. 
        #Esto no siempre es cierto por eso se prefiere obtener el año en el titulo
        elif len(lista) >1:
            valor = min(lista)
        else:
            valor = lista[0]
    #retonra el valor
    return valor

## Organizar Los Arhivos Oil

Esta funnción tiene como objetivo correr cada excel encontrar el año de cada archivo, encontrar donde estan ubicados sus encabezados y que encabezados tiene. Por ultimo, se encarga de ver si hay varios archivos del mismo año. Estos por lo general son archivos que estan en el ultimo año entonces solo llevan hasta un mes y van registrando todos los meses. Por Tanto el archivo que se quiere es el ultimo. Por ejemplo: se tienen varios archivos del 2020 del mes de enero hasta el mes de agosto. Este verifica cual es el ultimo es decir cual es agosto y ese es el que se va a leer ya que tiene todos los datos y previnimos de repetir datos y disminuir el tiempo. Como parametro se tiene lista de los nombres de archivo del crudo y retorna una lista de los años y del encabezado. Toca tener encuenta que la lista del año contiene un indice que dice cual de los archivos se van a leer. 

In [7]:
def orgazinarLosArchivosOil(listnameOil):
    listAnio = []
    listEncabezado = []
    #Este recorrido abre cada archivo.
    for i in range(0, len(listnameOil)):
        #abrir la hoja de excel
        loc = listnameOil[i]
        wb = xlrd.open_workbook(loc)
        sheet = wb.sheet_by_index(0)
        sheet.cell_value(0, 0)
        #encontrar encabezados y el año
        encabezado, indiceE = encontrarEncabezado(sheet)
        anio = encontrarAnio(sheet, indiceE, listnameOil[i])
        siHay = 0
        #Verificar si se repiten los años
        for j in range(0, len(listAnio)):
            #se encontrar y ahora buscar el mayor
            if anio == listAnio[j][0]:
                if len(encabezado)> len(listEncabezado[j][0]):
                    listAnio[j] = [anio,i]
                    listEncabezado[j] = [encabezado, indiceE]
                siHay = 1
        #si no entonces se guardan
        if siHay == 0:
            listAnio.append([anio, i])
            listEncabezado.append([encabezado, indiceE])
    return listAnio, listEncabezado


## Insertar Tabla Crudo 

La función tiene el proposito de insertar los datos en el SQL. Parametros, el año, la lista de encabezado, y la lista de los valores en cada columna. 

In [20]:
def insertarTablaCrudo(valor, encabezado, year):
    #Entra a la base de datos
    conn = sqlite3.connect("hackathon.db")
    #Inicializar variables
    c = conn.cursor()
    campo = ""
    departamento = ""
    municipio = ""
    operadora = ""
    cuenca = ""
    contrato = ""
    arregloMes = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    termino = False
    #Checkea si algun valor es nulo. Si lo es pone el valor como 0. 
    for i in range(0,len(encabezado)):
        if valor[i]=="":
            value = "0"
        else:
            value = valor[i]
        
        #Esto chekea que columna es la que se tiene y se va insertar en la base de datos.
        if "departamento" == encabezado[i].lower():
            departamento = value.lower()
        elif "campo" == encabezado[i].lower():
            campo = value.lower()
        elif "municipio" == encabezado[i].lower():
            municipio = value.lower()
            #Esto ya que archivos antes no habian operadora y empresa y asi no se sobrescribe y anh y blind funciona.
        elif ("operadora" == encabezado[i].lower() or "empresa" == encabezado[i].lower()) and operadora == "":
            operadora = value.lower()           
        elif "cuenca" == encabezado[i].lower():
            cuenca = value.lower()
        elif "contrato" == encabezado[i].lower():
            contrato = value.lower()
        elif "enero"== encabezado[i].lower() or "ene" in encabezado[i].lower():
            arregloMes[0] = float(value)
        elif "febrero"== encabezado[i].lower() or "feb" in encabezado[i].lower():
            arregloMes[1] = float(value) 
        elif "marzo"== encabezado[i].lower():
            arregloMes[2] = float(value)
        elif "abril"== encabezado[i].lower():
            arregloMes[3] = float(value)
        elif "mayo"== encabezado[i].lower():
            arregloMes[4] = float(value)
        elif "junio"== encabezado[i].lower():
            arregloMes[5] = float(value)
        elif "julio"== encabezado[i].lower():
            arregloMes[6] = float(value)
        elif "agosto"== encabezado[i].lower():
            arregloMes[7] = float(value)
        elif "septiembre"== encabezado[i].lower():
            arregloMes[8] = float(value)
        elif "octubre"== encabezado[i].lower():
            arregloMes[9] = float(value)
        elif "noviembre"== encabezado[i].lower():
            arregloMes[10] = float(value)
        elif "diciembre"== encabezado[i].lower():
            arregloMes[11] = float(value)
        elif "concatenar" == encabezado[i].lower():
            print("concatenar")
        else: 
            #Por si no es ninguna de las anteriores.
            print("no")
            print(encabezado[i].lower())
            print(valor[i])
    #Si termina pues no continua en lo contario inserta los datos
    if termino == True:
        print(termino) 
    else:
        conn = sqlite3.connect("hackathon.db")
        c = conn.cursor()
        #Se insertan los datos en cada uno de estos. 
        idD = insertarDepartamento(departamento)
        idM = insertarMunicipio(municipio)
        idE = insertarEmpresa(operadora)
        idC = insertarCuenca(cuenca)
        idCo = insertarContrato(contrato)
        idDa = insertarData(arregloMes)
        insertarCampoCrudo(idD, idM, idE, idC, idCo, idDa, campo, year)
        


## Insertar Borde ANH

Aqui tiene como objetivo marcar en la base de datos la separación entre ANH y los Blid data.

In [9]:
def insertarBordeANH():
    campo = "-1"
    departamento = "-1"
    municipio = "-1"
    operadora = "-1"
    cuenca = "-1"
    contrato = "-1"
    arregloMes = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
    year = "-1"
    idD = insertarDepartamento(departamento)
    idM = insertarMunicipio(municipio)
    idE = insertarEmpresa(operadora)
    idC = insertarCuenca(cuenca)
    idCo = insertarContrato(contrato)
    idDa = insertarData(arregloMes)
    insertarCampoCrudo(idD, idM, idE, idC, idCo, idDa, campo, year)


## Quitar Acento y Limpiar

Estas funciones tienen como proposito agarrar el valor a insertar y verificar si no tienen acento. En el caso que sí entonces les quita el acento con el fin de no causar errores. En el otro de limpiar verifica que no haya doble espacios es decir como: ECOPETROL__SAS deberia ser ECOPETROL_SAS (es sin el _ pero se pone para marcar el doble espacio). Tambien verifica que no haya espacio alfinal porque hay casos que estan asi. Estos son: CASANARE_ deberia ser solo CASANARA. Ambos les entra como parametro el vaor a guardar y retorna el valor modificado y arreglado. Estas dos funciones corren en runSQL. estan aqui para mostrar su funcionamiento. 

In [10]:
def quitarAcento(valor):
    #Acentos
    v = valor.replace("á","a")
    v2 = v.replace("é","e")
    v3 = v2.replace("í","i")
    v4 = v3.replace("ó", "o")
    v5 = v4.replace("ú", "u")
    return v5

def limpiar(valor):

    v = valor.replace("/"," ")
    v2 = v.replace("-"," ")
    v3 = v2.replace("  ", " ")
    v5 = v3.replace("  ", " ")
    v6 = v5.replace("  ", " ")
    v7 = v6.replace(".", "")
    
    v4 = ""
    #Quitar el espacio alfinal
    if len(v7)!=0:
        if " " == v7[len(v7)-1]:
            for i in range(0, len(v7)-1):
                v4+=v7[i]
        else:
            v4 = v7
    else:
        v4 = v7
    
    return v4

## Guardar Datos Oil :

Esta función se encarga de guardar los archivos. Parametro la lista de nombres de los archivos. 

In [11]:
def guardarDatosOil(listnameOil):
    #Organizar la lista de los archivos para retornar año y encabezado.
    anio, encabezado = orgazinarLosArchivosOil(listnameOil)
    print("starting to save the data in the database this might take some minutes")
    #Recorre cada año ya que la lista contiene el indice de los archivos que se van a leer y omitir repetidos.
    for i in range(0,len(anio)):
        indice = anio[i][1]
        year = anio[i][0]
        loc = listnameOil[indice]
        print("saving: ",loc)
        #abrir el excel
        wb = xlrd.open_workbook(loc)
        sheet = wb.sheet_by_index(0)
        sheet.cell_value(0, 0)
        #Aqui se recorre cada fila del archivo de excel.
        for m in range(encabezado[i][1]+1, sheet.nrows):
            #Se obtiene el valor de la fila con todas las columnas
            valor = sheet.row_values(m)
            #Se verifica si ya termino o no.
            if valor[0] == "" or valor[1] == "" or valor[2] == "" or "total" in valor[0].lower():
                break
            #Se insertan los datos en SQL 
            insertarTablaCrudo(valor,encabezado[i][0], anio[i][0])
        print("finish with: ", loc)

## Check if work:

Su objetivo es verificar si se descargaron todos los archivos del github. Retorna un booleano diciendo si verdadero o falso. Verdadero si si se leyeron todos los archivos falso si no.

In [12]:
def checkifwork():
    #Obtiene los nombres de los archivos de github.
    listnamefiles = [f for f in glob.glob("*.xlsx")]
    listnamefiles2 = [f for f in glob.glob("*.xls")]
    #Mira cuantos hay
    number = len(listnamefiles) + len(listnamefiles2)
    if number == 0:
        return False
    else:
        print(number," archivos descargado")
        print("please enter the number of Excelfiles that are in github, in order to check if it save all the files")
        print("if you are not sure how many files there are and you wish for it to run the local files then please write 100 and enter")
        x = input()
        result = False
        #checkea si funciono y si no.
        if number == int(x):
            print("the download was a success")
            result = True
        else:
            #Si no descargo los archivos los borra. 
            print("error downloading the files from github, please download the files manually and put them in the same folder of this file ")
            for i in range(0,len(listnamefiles)):
                os.remove(listnamefiles[i])
            for i in range(0,len(listnamefiles2)):
                os.remove(listnamefiles2[i])
        return result

## Delete Files :
   
Funcion es borrar los archivos que estan en la misma carpeta. Es decir borrar los archivos descargados no los locales. Entra la lista de archivos

In [13]:
def deleteFiles(listname):
    for i in range(0, len(listname)):
        #Remover archivos.
        os.remove(listname[i])

## Get Files Names GitHub:

El objetivo de esta funcion es obtener el url de los archivos a descargar. Retorna la lista de los url de los archivos

In [23]:
def getFilesNamesGitHub ():
    #url que se descarga
    url = 'https://github.com/dgguarin20/archivos2'
    #descargar file.txt
    html = requests.get(url)
    with open('file.txt', 'wb') as wr: 
        wr.write(html.content)
    wr.close()
    #abrir el archivo
    read = open("file.txt","r+",encoding="utf-8")

    listhtmlExcel = []
    htmlExcel = ""
    #buscar la linea que contiene la informacion de excel
    for line in read:
        xls = ".xls"
        if xls in line:
            htmlExcel = line
            listhtmlExcel.append(htmlExcel)

    listaExcel = []
    for j in range(0 , len(listhtmlExcel)):
        #El codigo en el file.txt se ve asi: <>NombreDelArchivo<href="URL.xls"> entonces se quiere obtener el URL.xls
        #Se elimina los y se dejan espacios para separar el nombre del archivo con href= "URL.xls"
        htmlExcel = listhtmlExcel[j]
        htmlExcel.replace(">", " ")
        htmlExcel.replace("<", " ")
        

        separacionExcel= np.array(htmlExcel.split(" "))
        for i in range(0,len(separacionExcel)):
            #se encontro el href para los archivos.
            if "href" in separacionExcel[i]:
                l = separacionExcel[i].split("\"")
                #se agregar la fuente principal para descargarlos
                urlgit = "https://github.com"+l[1]
                #se elimina blob por raw. ya que en github ellos guardan el enlace con raw no con blob.
                urlgit2 = urlgit.split("blob")
                urlgit = urlgit2[0]+"raw"+urlgit2[1]
                listaExcel.append(urlgit)
                
    read.close()
    listaExcel.append(" ")
    return listaExcel



## Write Output Data 

Esta función tiene el proposito de agarrar los datos de la base de datos y transformarlos en archivos excel con formato Long y Wide por preferencia para poder visuarlos mejor. Parametro es el nombre del archivo si van a ser los valores blind o los de la ANH. 

In [15]:
def writeOutputData(archivo):
    print("----------------------------------------")
    print("Creando Archivos Output para: " + archivo)
    print("----------------------------------------")
    #Crea el nombre del archivo y donde se va a guardar. En la carpeta output.
    url = 'Output\\'+archivo+'_long.xlsx'
    #abre el excel a escribir
    workbook = xlsxwriter.Workbook(url)
    worksheet = workbook.add_worksheet()
    
    #Pone el encabezado de cada columna.
    worksheet.write('A1', 'Campo') 
    worksheet.write('B1', 'Year') 
    worksheet.write('C1', 'Departamento') 
    worksheet.write('D1', 'Municipio')
    worksheet.write('E1', 'Empresa')
    worksheet.write('F1', 'Cuenca')
    worksheet.write('G1', 'Contrato')
    worksheet.write('H1', 'Mes')
    worksheet.write('I1', 'Produccion')
    Border = encontrarIDBorde()[0][0]
    campoInfo = getCampoCrudo()
    #Las letras son los indices que se manejan en excel
    letras = ['A','B','C','D','E','F','G','H','I']
    #ELos meses a tratar
    meses = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
    fila = 2
    n1 = 0
    l1 = Border-1
    #Identifica si son datos blind o del ANH. Estan guardados en diferentes indices entonces por eso el cambio.
    if archivo == 'Blind':
        n1 = Border
        l1 = len(campoInfo)
        
    #Recorrido de la base de datos hasta terminar.
    for i in range(n1, l1):
        #Agarra la informacion de la tabla de Campo que es la principal
        #En esta encuentra los id de cada otra tabla
        campoI = campoInfo[i]
        campo = campoI[1]
        year = campoI[2]
        #Tabla departamento y obtener su informacion es decir su nombre por su respectivo ID.
        departamento = getDepartamentoByValue("id",campoI[3])[0][1]
        #Tabla municipio y obtener su informacion es decir su nombre por su respectivo ID.
        municipio = getMunicipioByValue("id",campoI[4])[0][1]
        #Tabla Empresa y obtener su informacion es decir su nombre por su respectivo ID.
        operadora = getEmpresaByValue("id",campoI[5])[0][1]
        #Tabla Cuenca y obtener su informacion es decir su nombre por su respectivo ID.
        cuenca = getCuencaByValue("id",campoI[6])[0][1]
        #Tabla Contrato y obtener su informacion es decir su nombre por su respectivo ID.
        contrato = getContratoByValue("id", campoI[7])[0][1]
        #Tabla DATA y obtener su informacion es decir su nombre por su respectivo ID // este tiene los meses.
        data = getDataByValue("id", campoI[8])[0]
        #Poniendo los datos a guardar
        insertar = [campo, year, departamento, municipio, operadora, cuenca, contrato, "mes", 0]
        for j in range(1, len(data)):
            #Agarrar el de cada mes correspondiente
            mes = meses[j-1]
            insertar[7] = mes
            produccion = data[j]
            insertar[8] = produccion
            for s in range(0,len(letras)):
                #fila y columna
                letra = letras[s]+str(fila)
                #escribiendolo en el excel con su fila y columna correspondiente
                worksheet.write(letra, insertar[s])
            fila += 1
            
    workbook.close()
    
    #Ahora aqui se guarda el archivo wide
    url = 'Output\\'+archivo+'_wide.xlsx'
    #abre el excel a escribir
    workbook = xlsxwriter.Workbook(url)
    worksheet = workbook.add_worksheet()
    #Inicializa los encabezados
    worksheet.write('A1', 'Campo') 
    worksheet.write('B1', 'Year') 
    worksheet.write('C1', 'Departamento') 
    worksheet.write('D1', 'Municipio')
    worksheet.write('E1', 'Empresa')
    worksheet.write('F1', 'Cuenca')
    worksheet.write('G1', 'Contrato')
    letras = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S']
    meses = ['Enero','Febrero','Marzo','Abril','Mayo', 'Junio', 'Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
    #Pone el encabezado que es cada mes
    for i in range(0, len(meses)):
        num = i + 7 
        letra = letras[num] +'1'
        worksheet.write(letra, meses[i])
    #Identifica donde comenzar este es de runSQL
    Border = encontrarIDBorde()[0][0]
    campoInfo = getCampoCrudo()
    
    n1 = 0
    l1 = Border-1
    #verifica si es blind o no.
    if archivo == 'Blind':
        n1 = Border
        l1 = len(campoInfo)  
    #Recorrido a guardar
    for i in range(n1, l1):
        campoI = campoInfo[i]
        campo = campoI[1]
        year = campoI[2]
        departamento = getDepartamentoByValue("id",campoI[3])[0][1]
        municipio = getMunicipioByValue("id",campoI[4])[0][1]
        operadora = getEmpresaByValue("id",campoI[5])[0][1]
        cuenca = getCuencaByValue("id",campoI[6])[0][1]
        contrato = getContratoByValue("id", campoI[7])[0][1]
        data = getDataByValue("id", campoI[8])[0]
        insertar = [campo, year, departamento, municipio, operadora, cuenca, contrato]
        #aqui guarda la informacion de campo hasta contrato
        for j in range(0, len(insertar)):
            num = i+2
            letra = letras[j]+str(num)
            worksheet.write(letra, insertar[j])
        #aqui escribe las columnas de los meses
        for j in range(1, len(data)):
            letra = letras[j+6]+str(num)
            worksheet.write(letra, data[j])
    
    print('Se crearon los Output')
    workbook.close()

## Run File

Esta función corre todas las funciones necesarias para descargar y guardar la informacion de ANH y los blind files. 

In [16]:
def RunFile():
    print("WELCOME!")
    print("BEFORE STARTING CHECK YOU DONT HAVE ANY EXCEL FILES IN THIS FOLDER, ONCE YOU ARE READY CLICK ENTER")
    x = input()
    print("FIRST OF ALL LETS START WITH THE ANH FILES FROM THE WEBPAGE")
    #obtener nombres y guardarlos
    #La pagina ANH no funciona
    #listaExcel = getFilesNames()
    #downloadFiles (listaExcel)
    listnameGas, listnameOil, listnameAll = getGasOilFileNames(False)
    print(".....................")
    print("CREATING THE DATABASE")
    print(".....................")
    #Crear base de datos
    create_tables()
    #Guardar loS datos
    guardarDatosOil(listnameOil)
    #Insertar en la base de datos donde estan ANH y los Blind
    insertarBordeANH()
    print("FINISH DOWNLOADING ALL THE ANH FILES IN THE DATABASE")
    print("------------------------------------------------")
    print("DELETING THE EXCEL FILES")
    #Borar los datos descargados
    deleteFiles(listnameAll)
    print("------------------------------------------------")
    print("DOWNLOADING BLIND FILES FROM GITHUB:")
    #buscar los archivos en git que son los blind y descargarlos
    namefilesGithub = getFilesNamesGitHub()
    downloadFiles(namefilesGithub)
    #checkear si funciona
    result = checkifwork()
    #Guardar los datos
    listnameGas, listnameOil, listnameAll = getGasOilFileNames(result)
    guardarDatosOil(listnameOil)
    writeOutputData('ANH')
    writeOutputData('Blind')
    deleteFiles(listnameAll)
    print("THANK YOU FOR WAITING!")
    print("WE ARE DONE PLEASE RUN THE OTHER PYTHON FILES TO SEE THE RESULTS THANK YOU")


In [24]:
RunFile()

WELCOME!
BEFORE STARTING CHECK YOU DONT HAVE ANY EXCEL FILES IN THIS FOLDER, ONCE YOU ARE READY CLICK ENTER

FIRST OF ALL LETS START WITH THE ANH FILES FROM THE WEBPAGE
.....................
CREATING THE DATABASE
.....................
starting to save the data in the database this might take some minutes
saving:  Files\Produccion-fiscalizada-crudo-2017.xlsx
finish with:  Files\Produccion-fiscalizada-crudo-2017.xlsx
saving:  Files\Producción Fiscalizada Crudo 2015.xlsx
finish with:  Files\Producción Fiscalizada Crudo 2015.xlsx
saving:  Files\Producción Fiscalizada Crudo 2018.xlsx
finish with:  Files\Producción Fiscalizada Crudo 2018.xlsx
saving:  Files\Producción Fiscalizada Crudo 2019-DIC.xlsx
finish with:  Files\Producción Fiscalizada Crudo 2019-DIC.xlsx
saving:  Files\Producción Fiscalizada Crudo 2020 Agosto.xlsx
finish with:  Files\Producción Fiscalizada Crudo 2020 Agosto.xlsx
saving:  Files\Producción fiscalizada de crudo año 2013.xlsx
finish with:  Files\Producción fiscalizada de 